In [ ]:
import CRDCLib as crdc
import pandas as pd

In [2]:
mdf_props_file = r"C:\Users\pihltd\Downloads\ctdc_model_properties_file.yaml"
columns = ['Field','ID', 'Version', 'Source', 'caDSR Name']
cde_df = pd.DataFrame(columns=columns)

In [3]:
propsjson = crdc.readYAML(mdf_props_file)
propsjson = propsjson['PropDefinitions']
print(propsjson)

{'crdc_id': {'Desc': 'The crdc_id is a unique identifier that is generated by Data Hub', 'Type': 'string', 'Tags': {'Template': 'No'}}, 'program_name': {'Desc': '<The narrative title used to refer to a broad framework (an administrative umbrella) of goals under which related projects or other research activities are grouped. Example - Clinical Proteomic Tumor Analysis.> <br>CDE ID = 11444542', 'Term': [{'Origin': 'caDSR - CRDC', 'Code': '11444542', 'Value': 'Program Name Text', 'Version': '1'}], 'Src': 'DSS', 'Type': 'string', 'Req': 'Yes'}, 'program_short_name': {'Desc': '<An acronym or abbreviated form of the title of a broad framework of goals under which related projects or other research activities are grouped. For example, CPTAC.> <br>CDE ID = 11459801 <br>This property is used as the key via which child records, e.g. project and/or study/trial records, can be associated with the appropriate program during data loading, and to identify the correct records during data updates.', '

In [4]:
for prop, value in propsjson.items():
    #print(value)
    if 'Term' in value:
        for entry in value['Term']:
            cdeid = entry['Code']
            cdeversion = entry['Version']
            cdesource = entry['Origin']
            cdeinfo = crdc.getCDERecord(cdeid,cdeversion)
            if cdeinfo['status'] == 'success':
                cdename = cdeinfo['DataElement']['longName']
                cde_df.loc[len(cde_df.index)] = [prop, cdeid, cdeversion, cdesource, cdename]
            else:
                cde_df.loc[len(cde_df.index)] = [prop, cdeid, cdeversion, cdesource, 'Not a caDSR CDE']

In [5]:
print(cde_df.head())

                Field        ID Version        Source               caDSR Name
0        program_name  11444542       1  caDSR - CRDC        Program Name Text
1  program_short_name  11459801       1  caDSR - CRDC  Program Short Name Text
2        project_name  11459804       1  caDSR - CRDC        Project Name Text
3  project_short_name  11459806       1  caDSR - CRDC  Project Short Name Text
4          study_name  11459810       1  caDSR - CRDC          Study Name Text


In [6]:
noncadsr_df = cde_df[~cde_df['Source'].str.contains('caDSR')]
print(noncadsr_df.head())

                  Field       ID Version               Source  \
9     person_first_name  2179589       2  CSAERS/NCI Standard   
10     person_last_name  2179591       2  CSAERS/NCI Standard   
11   person_middle_name  2179590       2  CSAERS/NCI Standard   
46    ctep_disease_term  4723846       1             THERADEX   
47  meddra_disease_code  2004425       4               ADEERS   

                                           caDSR Name  
9                             Person Given/First Name  
10                            Person Family/Last Name  
11                                 Person Middle Name  
46  NCI CTEP Simplified Disease Classification Ter...  
47                                        MedDRA Code  


In [7]:
outfile = r"C:\Users\pihltd\Downloads\ctdc_non_cadsr.tsv"
noncadsr_df.to_csv(outfile, sep="\t")